# Confiabilidade de viga mista

## Importando bibliotecas

In [562]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt
import pandas as pd

## Tabela de perfis

In [563]:
tabela = pd.read_excel('Catalogo-Tabela-Bitolas-Editado-2.xlsx')
display(tabela)

,massa_linear,d,bf,tw,tf,h,d',area,Ix,Wx,...,ly,Wy,ry,Zy,rt,lt,Mesa,Alma,Cw,u\n
0,13.0,148,100,4.3,4.9,138,118,16.6,635,85.8,...,82,16.4,2.22,25.5,2.60,1.72,10.20,27.49,4181,0.67
1,18.0,153,102,5.8,7.1,139,119,23.4,939,122.8,...,126,24.7,2.32,38.5,2.69,4.34,7.18,20.48,6683,0.69
2,22.5,152,152,5.8,6.6,139,119,29.0,1229,161.7,...,387,50.9,3.65,77.9,4.10,4.75,11.52,20.48,20417,0.88
3,24.0,160,102,6.6,10.3,139,115,31.5,1384,173.0,...,183,35.9,2.41,55.8,2.73,11.08,4.95,17.48,10206,0.69
4,29.8,157,153,6.6,9.3,138,118,38.5,1739,221.5,...,556,72.6,3.80,110.8,4.18,10.95,8.23,17.94,30277,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,154.2,623,229,14.0,24.9,573,541,196.5,125783,4038.0,...,4999,436.6,5.04,683.3,5.94,303.29,4.60,38.66,4456995,2.11
104,155.0,611,324,12.7,19.0,573,541,198.1,129583,4241.7,...,10783,665.6,7.38,1022.6,8.53,200.77,8.53,42.60,9436714,2.47
105,174.0,616,325,14.0,21.6,573,541,222.8,147754,4797.2,...,12374,761.5,7.45,1171.1,8.58,286.88,7.52,38.63,10915665,2.48
106,195.0,622,327,15.4,24.4,573,541,250.1,168484,5417.5,...,14240,870.9,7.55,"1341 ,0",8.66,405.29,6.70,35.14,12695302,2.49


## Definindo funções

In [564]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    AMOSTRAS: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [565]:
espessura_laje: float = 10 # cm 
desvpad_espessura_laje: float = 0.06 * espessura_laje # cm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [566]:
limite_escoamento_perfil: float = 250 * 1.08 # MPa
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil # MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [567]:
limite_escoamento_armadura: float = 500 * 1.08 # MPa
desvpad_escoamento_armadura: float = 0.05 * limite_escoamento_armadura #MPa

limites_escoamento_armadura: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [568]:
resistencia_concreto: float = 20 * 1.17 # MPa
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto # MPa

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [569]:
diametro_rebite: float = 1.59 # cm
desvpad_diametro_rebite: float = 0.01 * diametro_rebite # cm

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [570]:
resistencia_rebite: float = 415 * 1.06 # MPa
desvpad_resistencia_rebite: float = 0.06 * resistencia_rebite # MPa

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [571]:
carga_permanente_g1: float = 7.6 # kN/m
carga_permanente_g2: float = 5 # kN/m

variacao_carga: list = [3, 11]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9 # m
espacamento_laje: float = 2.8 # m

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * (vao_laje ** 2)) / 8 for i in range(len(carga_distribuida_projeto))]

altura_perfil_ht: float = 45 # cm
coeficiente_ya1: float = 1.1
x: float = 10 # cm

# Tabela com áreas filtradas a partir de 450 mm
tabela_filtrada = tabela.loc[tabela['d'] >= 450]

row_tabela: int = 0

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
area_minima_inicial: float = tabela_filtrada.iloc[0]['area']
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_perfil_ht / 2) + espessuras_laje[j] - (x / 2)))) # cm²
        if (areas_minimas[index_matriz][j] > area_minima_inicial):
            print(f'Área mínima inicial: {area_minima_inicial}')
            print(f'Área mínima calculada: {areas_minimas[index_matriz][j]}')
            row_tabela += 1
            area_minima_inicial = tabela_filtrada.iloc[row_tabela]['area']
    index_matriz += 1
    

print(f'Áreas mínimas: {areas_minimas}')
print(f'Área mínima final: {area_minima_inicial}')


Área mínima inicial: 66.6
Área mínima calculada: 66.71323879437502
Áreas mínimas: [[26.796204693482274, 29.448571522621357, 29.986045869215932, 27.76104877765678, 32.31803796848347, 32.64411934684089, 29.106746226380803, 30.298406883190765, 29.648908716980518, 29.798528300036914, 32.54124669380363, 25.860227165647505, 27.26737552900783, 26.540099597818386, 34.88113658252039, 28.8265312199466, 32.18288209661507, 29.444562701694704, 29.882164277778422, 30.96466578519739, 30.10091932240509, 29.76149679215465, 30.74535454822171, 36.17303067758429, 31.89195190474841, 30.25135607670889, 31.892561429262695, 29.838330316771113, 27.320582312681978, 30.34307754785952, 30.14330377822785, 30.55700863772272, 29.711616762611705, 31.06980295417093, 28.248786071465346, 27.389912561104328, 25.381339507960746, 29.821519642379265, 27.536738570261544, 33.050465246416664, 30.284459550328705, 31.155962126944093, 25.32959364255248, 29.69362334563773, 28.785122375798192, 30.37962598755638, 29.723996932987514,

## Flambagem da seção

In [572]:
# Classificação da seção quanto a flambagem
altura_perfil: float = tabela_filtrada.iloc[row_tabela]['d\''] # mm
altura_total_perfil: float = tabela_filtrada.iloc[row_tabela]['d'] # mm
altura_efetiva_perfil: float = tabela_filtrada.iloc[row_tabela]['h'] # mm
espessura_perfil: float = tabela_filtrada.iloc[row_tabela]['tw'] # mm
espessura_mesa: float = tabela_filtrada.iloc[row_tabela]['tf'] # mm
largura_mesa: float = tabela_filtrada.iloc[row_tabela]['bf'] # mm
modulo_elasticidade_perfil: float = 200000 # MPa
print(f'Altura perfil: {altura_perfil}, Altura total do perfil: {altura_total_perfil} e Espessura do perfil: {espessura_perfil}')

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

contador_compacto: int = 0
contador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        contador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        contador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {contador_compacto}')
print(f'Semi-compactas: {contador_semicompacto}')
        

Altura perfil: 404, Altura total do perfil: 455 e Espessura do perfil: 8.0
Compactas: 1000
Semi-compactas: 0


## Momento resistência da viga mista

In [573]:
# Resistência à compressão do concreto

yc: float = 1.4
largura_efetiva_laje: float = 225 # cm

rcd: list = []

for i in range(len(resistencias_concreto)):
    rcd.append(0.85 * (resistencias_concreto[i] / 10) * largura_efetiva_laje * espessuras_laje[i] / yc)

print(f'Rcd: {rcd}')

# Resistência à tração do aço

rtd: list = []

for i in range(len(limites_escoamento_perfil)):
    rtd.append(area_minima_inicial * (limites_escoamento_perfil[i] / 10) / coeficiente_ya1)
    
print(f'Rtd: {rtd}')

profundidades_linha_neutra: list = []
momentos_resistente_laje_concreto: list = []

for i in range(len(rtd)):
    if rcd[i] >= rtd[i]:
        aux = rtd[i]/(rcd[i] / espessuras_laje[i])
        profundidades_linha_neutra.append(aux)
        momentos_resistente_laje_concreto.append(rtd[i] * ((altura_total_perfil / 20) + espessuras_laje[i] - aux / 2) / 100)
    else:
        cad: float = 0.5 * (rtd[i] - rcd[i])
        print(f'Cad: {cad}')
        intervalo = (limites_escoamento_perfil[i] / 10) * (largura_mesa / 10) * (espessura_mesa / 10) / coeficiente_ya1
        if cad < intervalo:
            yp: float = cad / ((limites_escoamento_perfil[i] / 10) * (largura_mesa / 10) / coeficiente_ya1)
            print(f'yp -> Cad MENOR intervalo: {yp}')
        else:
            yp: float = ((cad - intervalo) / ((limites_escoamento_perfil[i] / 10) * (espessura_perfil / 10) / coeficiente_ya1)) + (espessura_mesa / 10)
            print(f'yp -> Cad MAIOR intervalor: {yp}')
            
        area_tracionada_aco: float = area_minima_inicial - (largura_mesa / 10) * yp
        print(f'Área tracionada do aço: {area_tracionada_aco}')
        
        centro_gravidade_area_tracionada: float = ((((largura_mesa / 10) * ((espessura_mesa / 10) ** 2)) / 2) + (altura_efetiva_perfil / 10) * (espessura_perfil / 10) * ((altura_efetiva_perfil / 10) / 2 + (espessura_mesa / 10)) + (largura_mesa / 10) * ((espessura_mesa / 10) - yp) * (((espessura_mesa / 10) - yp) / 2 + (altura_efetiva_perfil / 10) + (espessura_mesa / 10))) / area_tracionada_aco
        print(f'Centro de gravidade da área tracionada: {centro_gravidade_area_tracionada}')
        
        momento_calculado: float = rcd[i] * (altura_total_perfil / 10 - centro_gravidade_area_tracionada + espessuras_laje[i] / 2) + cad * (altura_total_perfil / 10 - centro_gravidade_area_tracionada - yp / 2)
        momentos_resistente_laje_concreto.append(momento_calculado / 100)
        print(f'Momento calculado: {momento_calculado / 100}')
        
print(f'x: {profundidades_linha_neutra}')
print(f'Momentos resistentes da laje de concreto:\n{momentos_resistente_laje_concreto}')



Rcd: [3813.364467105863, 2648.5891665364475, 2850.6164933521245, 3262.1905983464885, 3498.7086533503975, 2782.567089058969, 3346.8711679392586, 3098.7545404220637, 2641.712875962364, 2943.261460343969, 3652.7234008736737, 3410.9016230560646, 3616.789102762696, 2540.0610371601147, 3411.911259452924, 3274.7510576536442, 3211.7946011729396, 2847.6201737023985, 2850.914820989687, 2597.972500158521, 3596.9054828804055, 1798.3039377063603, 3328.871261110161, 4148.9515946135225, 3298.116714234383, 2764.4109303036967, 3987.97975788451, 2513.223731678483, 3697.030357005948, 3410.714150024847, 3176.7472953406573, 3692.9682472201052, 2625.1930246584307, 3198.928811241603, 2813.978417905563, 3294.7382610875857, 3531.2667765939304, 3107.8371192460463, 3372.949325382323, 2919.760849453339, 3795.774486164469, 4348.193163673347, 2750.7962908213576, 4018.539016429218, 3288.35000776753, 3717.393797132942, 2873.224206795967, 2465.2145575433046, 3327.821634418798, 3362.6247174769483, 2518.2245079518784, 4

## Confiabilidade do momento resistente

In [574]:
# Cálculo da probabilidade de falha

probabilidades_falhas: list = []

for i in momento_solicitante_projeto:
    print(i)
    falhas: int = 0
    for j in range(len(momentos_resistente_laje_concreto)):
        print(f'Momento solicitante: {i} e Momento resistente: {momentos_resistente_laje_concreto[j]}')
        if momentos_resistente_laje_concreto[j] - i < 0:
            falhas += 1
    probabilidades_falhas.append(falhas / len(momentos_resistente_laje_concreto))

print(falhas)
print(f'Probabilidades de falha: {probabilidades_falhas}')
    

194.8773214285714
Momento solicitante: 194.8773214285714 e Momento resistente: 601.1686950478594
Momento solicitante: 194.8773214285714 e Momento resistente: 541.9437938357221
Momento solicitante: 194.8773214285714 e Momento resistente: 531.8164253612424
Momento solicitante: 194.8773214285714 e Momento resistente: 575.6863911491506
Momento solicitante: 194.8773214285714 e Momento resistente: 509.2472032214988
Momento solicitante: 194.8773214285714 e Momento resistente: 492.8863985139253
Momento solicitante: 194.8773214285714 e Momento resistente: 558.3265082436983
Momento solicitante: 194.8773214285714 e Momento resistente: 532.9554387510608
Momento solicitante: 194.8773214285714 e Momento resistente: 538.3377382654502
Momento solicitante: 194.8773214285714 e Momento resistente: 540.3281138229383
Momento solicitante: 194.8773214285714 e Momento resistente: 505.5114561024469
Momento solicitante: 194.8773214285714 e Momento resistente: 622.3963824811372
Momento solicitante: 194.877321428